## 그룹 연산은 데이터를 집계하거나 변환하는 등의 작업을 한 번에 처리할 수 있는 기능

## groupby 메서드로 평균값 구하기

In [3]:
import pandas as pd
df = pd.read_csv('../data/gapminder.tsv', sep = '\t')

df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [7]:
# groupby 메서드를 사용해 = year 열 기준, lifeExp 열의 연도별 평균값을 구함

avg_life_exp_by_year = df.groupby('year').lifeExp.mean()

print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 분할-반영-결합 과정 살펴보기 - groupby 메서드
* groupby 메서드를 분해하는 것이 불가능하므로 비슷한 연산을 수행

In [8]:
# groupby 메서드에 열 이름을 전달하면 '분할' 작업 발생

years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [10]:
# 연도별로 데이터 추출 : 1952년을 추출한 예, "반영-1" 과정

y1952 = df.loc[df.year == 1952, :]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [11]:
# 추출한 데이터의 평균을 구함 '반영 -2'

y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)


49.05761971830987


In [13]:
# 연도별 평균값을 구함 - '반영 - 3'

y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :]
y1962_mean = y1962.lifeExp.mean()
print(y1962_mean)

y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()
print(y2007_mean)

51.50740112676054
53.60924901408449
67.00742253521126


In [14]:
# 연도별로 계산한 lifeExp의 평균값을 합침 - '결합'

df2 = pd.DataFrame({"year":[1952, 1957, 1962, 2007],
                  "":[y1952_mean, y1957_mean, y1962_mean, y2007_mean]})

print(df2)

   year           
0  1952  49.057620
1  1957  51.507401
2  1962  53.609249
3  2007  67.007423


## 평균값을 구하는 사용자 함수와 groupby 메서드

In [1]:
def my_mean(values):
    n = len(values)
    print('n -> %d \n' %n)
    
    sum = 0
    for value in values:
        print('value -> %d' %value)
        sum += value # sum = sum + value
        print('sum -> %d \n' %sum)
        
    return sum / n

In [39]:
my_mean([1,2,4,7])

n -> 4 

value -> 1
sum -> 1 

value -> 2
sum -> 3 

value -> 4
sum -> 7 

value -> 7
sum -> 14 



3.5

In [2]:
# 사용자 정의 함수(my_mean)를 groupby 메서드와 조합하기 위해 agg 메서드를 사용

agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

NameError: name 'df' is not defined

In [43]:
# 결론적으로 mean메서드를 사용하여 얻은 평균과 같다.

agg_my_mean1 = df.groupby('year').lifeExp.mean()
agg_my_mean1

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

## 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [50]:
# 첫 번째 인자로 받은 값의 평균을 구하여 두 번째 인자로 받은 값과 차이를 계산
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    #print('mean -> %.1f' %mean)
    return mean - diff_value

In [51]:
my_mean_diff([1,2,3,4], 10)

-7.5

In [52]:
# 전체 평균 수명

global_mean = df.lifeExp.mean()
print(global_mean)

59.47443936619713


In [53]:
# 첫 번째 인자에 my_mean_diff, 두 번째 인자에 전체 평균 수명값 전달
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [96]:
#문제 - 알아보기 쉽게 결과를 나타내 보세요.

agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean) 
a = pd.DataFrame(agg_my_mean1)
a['global_mean'] = 59.47443936619713
b = pd.DataFrame(agg_mean_diff)
result = pd.concat([a, b], axis = 1)
result.columns =['연도별 평균 수명', '전체 평균 수명','결과']
result = result.reset_index()
result


,year,연도별 평균 수명,전체 평균 수명,결과
0,1952,49.057620,59.474439,-10.416820
1,1957,51.507401,59.474439,-7.967038
2,1962,53.609249,59.474439,-5.865190
3,1967,55.678290,59.474439,-3.796150
4,1972,57.647386,59.474439,-1.827053
5,1977,59.570157,59.474439,0.095718
6,1982,61.533197,59.474439,2.058758
7,1987,63.212613,59.474439,3.738173
8,1992,64.160338,59.474439,4.685899
9,1997,65.014676,59.474439,5.540237


## 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [98]:
# 집계 메서드를 리스트에 담아 전달하기
# 연도별로 그룹화한 lifeExp열의 0이 아닌 값의 갯수, 평균, 표준편차 담아 전달

import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


In [101]:
# 집계 메서드를 딕셔너리에 담아 agg 메서드에 전달

gdf_dict = df.groupby('year').agg({
    'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'})

print(gdf_dict)       

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


# 데이터 변환

## 표준점수 계산하기

In [102]:
# 통계 : 표준점수 = 데이터 평균 - 표준편차

def my_zscore(x):
    return (x - x.mean()) / x.std()

In [104]:
# transform 데이터 변환 메서드
# 데이터와 메서드를 일대일로 대응시켜 계산
# 데이터의 양이 줄어들지 않는다 - 데이터 변환하는데 사용

transform_z = df.groupby('year').lifeExp.transform(my_zscore)

print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [105]:
# 데이터의 양이 줄어들지 않음을 확인
print(df.shape)

(1704, 6)


In [106]:
# 데이터의 양이 줄어들지 않음을 확인 (1704로 원본 데이터와 동일)
print(transform_z.shape)

(1704,)


In [111]:
# 데이터의 양이 줄어들지 않는다- 데이터 변환(transform)하는데 사용

a = df.head(2)
print(a)
print('\n-----------------------------------------')
b = df.groupby('year').lifeExp.mean().head(2)
print(b)
print('\n-----------------------------------------')
c = df.groupby('year').lifeExp.std().head(2)
print(c)
print('\n-----------------------------------------')
print((28.801 - 49.057620)/ 12.225956)
print((30.332  - 51.507401)/  12.231286)

       country continent  year  lifeExp      pop   gdpPercap
0  Afghanistan      Asia  1952   28.801  8425333  779.445314
1  Afghanistan      Asia  1957   30.332  9240934  820.853030

-----------------------------------------
year
1952    49.057620
1957    51.507401
Name: lifeExp, dtype: float64

-----------------------------------------
year
1952    12.225956
1957    12.231286
Name: lifeExp, dtype: float64

-----------------------------------------
-1.656853664449635
-1.7312489463495497


## 누락값을 평균값으로 처리하기

In [113]:
import seaborn as sns
import numpy as np

# sample()을 활용한 무작위 샘플 데이터 만들기
# 무작위 표본 추출을 하는데 np.random.permutation() 함수를 사용해서 순열을 무작위로
# 뒤섞은 후에 n개 만큼 indexing 해 오는 방법을 사용

# total_bill 열의 값 4개를 임의로 선택해서 누락값으로 바꿈

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN

print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [114]:
# total_bill 4개 누락값이므로 6개만 나옴
# 누락값 - 여성1명 , 남성 3명
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [115]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [117]:
# 성별을 구분하여 total_bill 열의 평균을 구하는 함수를 적용

total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


In [126]:
tips_10.groupby('sex').mean()

,total_bill,tip,size,fill_total_bill
sex,,,,
Male,17.9525,2.875714,2.571429,17.9525
Female,13.9300,2.506667,2.000000,13.9300


## 데이터 필터링 사용하기 - filter 메서드

In [128]:
tips = sns.load_dataset('tips')
print(tips)
print(tips.shape)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]
(244, 7)


In [129]:
# size 열의 데이터 수를 확인
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [139]:
# 주문이 30번 이상 있는 테이블만 그룹화

tips_filtered = tips.groupby('size').\
                filter(lambda x: x['size'].count() >= 30)

In [140]:
print(tips_filtered.shape)

(231, 7)


In [141]:
print(tips_filtered['size'].value_counts())

2    156
3     38
4     37
Name: size, dtype: int64


## 그룹 오브젝트 저장하여 살펴보기

In [142]:
tips_10 = sns.load_dataset('tips').sample(10, random_state = 42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [143]:
grouped = tips_10.groupby('sex')
print(grouped)

In [144]:
# 그룹 오브젝트에 포함된 그룹을 보려면 groups 속성을 출력

print(grouped.groups)

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'), 'Female': Int64Index([198, 124, 101], dtype='int64')}


## 그룹 오브젝트의 평균 구하기

In [146]:
# 그룹 오브젝트를 이용하여 평균을 구함
# smoker, day, time 열은 그룹 연산에서 제외됨
# 파이썬은 그룹 연산에 적합한 열을 알아서 골라줌

avgs = grouped.mean()
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [147]:
print(tips_10.columns)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


## 그룹 오브젝트에서 데이터 추출하고 반복하기

In [149]:
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


## 그룹 오브젝트 계산하고 살펴보기

In [151]:
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [152]:
print(type(group_avg))

<class 'pandas.core.frame.DataFrame'>


In [153]:
print(group_avg.columns)

Index(['total_bill', 'tip', 'size'], dtype='object')


In [154]:
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [156]:
# reset_index 메서드를 사용하여 데이터프레임의 인덱스를 새로 부여

group_method = tips_10.groupby(['sex', 'time']).mean().reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [157]:
# reset_index 메서드 대신 as_index=False 를 설정해도 같은 결과가 나옴

group_param = tips_10.groupby(['sex', 'time'], as_index = False).mean()
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
